##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Сохранение и сериализация моделей с Keras

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras/save_and_serialize"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/guide/keras/save_and_serialize.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/guide/keras/save_and_serialize.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/guide/keras/save_and_serialize.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

Первая часть руководства охватывает сохранение и сериализацию для Sequential и построенных с помощью Functional API моделей. API сохранения и сериализации в точности совпадают для обеих видов моделей.

Сохранение для кастомных субклассов `Model` рассматривается в разделе "Сохранение наследованных моделей". В этом случае API немного отличается от Sequential или Functional моделей.

## Установка

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

tf.keras.backend.clear_session()  # Для легкого сброса состояния ноутбука.

## Часть I: Сохранение Sequential моделей Functional моделей

Давайте рассмотрим следующую модель:

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='3_layer_mlp')
model.summary()

Хоть это и необязательно, давайте обучим эту модель, чтобы она имела веса и состояние оптимизатора которые можно сохранить.
Конечно вы можете сохранить модель, которую вы не обучали, но очевидно это не так интересно.

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop())
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=1)

In [ ]:
# Сохраняем прогнозы для будущих проверок
predictions = model.predict(x_test)

### Сохранение всей модели

Вы можете сохранить модель построенную с Functional API в один файл. Позже вы можете восстановить ту же модель из этого файла, даже если у вас больше нет доступа к исходному коду.

Этот файл включает:

- Архитектуру модели
- Значения весов модели (которые были получены во время обучения)
- Конфигурацию обучения модели (то, что вы передаете `compile`), если таковая есть
- Оптимизатор и его состояние, если имеется (это позволит вам продолжить обучение с места где вы остановились)

In [ ]:
# Сохранение модели
model.save('path_to_my_model.h5')

# Восстановление в точности той же модели из файла
new_model = keras.models.load_model('path_to_my_model.h5')

In [ ]:
import numpy as np

# Проверим что состояние сохранилось
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Заметим, что состояние оптимизатора также сохранилось:
# вы можете продолжить обучение с места где вы остановились.

### Экспорт в SavedModel

Вы можете также экспортировать всю модель в формат TensorFlow `SavedModel`. `SavedModel` является самостоятельным форматом сериализации для объектов TensorFlow, поддерживаемым TensorFlow serving, а также отличным от Python реализациям TensorFlow.

In [ ]:
# Экспортируем модель в SavedModel
model.save('path_to_saved_model', save_format='tf')

# Восстанавливаем в точности ту же модель
new_model = keras.models.load_model('path_to_saved_model')

# Проверяем что состояние сохранилось
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Заметим, что состояние оптимизатора также сохранилось:
# вы можете продолжить обучение с места где вы остановились.

Файлы `SavedModel` которые мы создали содержат:

- Чекпоинт TensorFlow содержащий веса модели.
- Прототип `SavedModel` содержащий граф TensorFlow лежащий в основе модели.

### Сохранение только архитектуры

Иногда вас интересует только архитектураа модели, и вам не нужно сохранять значения весов или оптимизатор. В этом случае вы можете получить "config" модели с помощью метода `get_config()`. Конфиг является словарем Python который позволяет вам воссоздать ту же модель -- инициализированную с нуля, не содержащую никакой информации полученнной ранее в результате обучения.

In [ ]:
config = model.get_config()
reinitialized_model = keras.Model.from_config(config)

# Заметьте что состояние модели не сохранено! Мы сохранили только архитектуру.
new_predictions = reinitialized_model.predict(x_test)
assert abs(np.sum(predictions - new_predictions)) > 0.

В качестве альтернативы вы можете использовать `to_json()` и `from_json()`, которые используют строку JSON для хранения конфигурации вместо словаря Python. Это полезно при сохранении конфигурации на диск.

In [ ]:
json_config = model.to_json()
reinitialized_model = keras.models.model_from_json(json_config)

### Сохранение только весов

Иногда вас интересует только состояние модели -- значение ее весов -- а не архитектура. В этом случае вы можете получить значения весов в виде списка массивов Numpy с помощью `get_weights()`, и установить состояние модели с помощью `set_weights`:

In [ ]:
weights = model.get_weights()  # Получает состояние модели.
model.set_weights(weights)  # Устанавливает состояние модели.

Вы можете комбинировать `get_config()`/`from_config()` и `get_weights()`/`set_weights()` для восстановления вашей модели в том же состоянии. Однако, в отличие от `model.save()` тут не включены конфигуарция обучения и оптимизатор. Вам нужно вновь вызвать `compile()` перед использованием модели для обучения.

In [ ]:
config = model.get_config()
weights = model.get_weights()

new_model = keras.Model.from_config(config)
new_model.set_weights(weights)

# Проверяем что состояние сохранено
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Заметим что оптимизатор не сохранился,
# поэтому модель должна быть вновь скомпилирована перед обучением
# (и оптимизатор начнет с пустого состояния).

Альтернативой `get_weights()` и `set_weights(weights)` для работы с диском
являются `save_weights(fpath)` и `load_weights(fpath)`.

Вот пример сохранения на диск:

In [ ]:
# Сохраним конфигурацию JSON на диск
json_config = model.to_json()
with open('model_config.json', 'w') as json_file:
    json_file.write(json_config)
# Сохраним веса на диск
model.save_weights('path_to_my_weights.h5')

# Загрузим модель из 2 файлов которые мы сохранили
with open('model_config.json') as json_file:
    json_config = json_file.read()
new_model = keras.models.model_from_json(json_config)
new_model.load_weights('path_to_my_weights.h5')

# Проверим, что состояние сохранилось
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Заметим, что оптимизатор не сохранился.

Но запомните, что самый простой и рекомендуемый способ следующий:

In [ ]:
model.save('path_to_my_model.h5')
del model
model = keras.models.load_model('path_to_my_model.h5')

### Сохранение только весов с использованием чекпоинтов TensorFlow

Заметьте, что `save_weights` может создавать файлы как в формате Keras HDF5,
так и в [формате TensorFlow Checkpoint](https://www.tensorflow.org/api_docs/python/tf/train/Checkpoint). Формат подразумевается из расширения вашего файла: если это ".h5" или ".keras", фреймворк использует формат Keras HDF5. Все остальное по умолчанию сводится к Checkpoint.

In [ ]:
model.save_weights('path_to_my_tf_checkpoint')

Для полной ясности формат может быть явно передан через аргумент `save_format` , который может принимать значение "tf" или "h5":

In [ ]:
model.save_weights('path_to_my_tf_checkpoint', save_format='tf')

## Сохранение наследованных моделей

Sequential и Functional модели это структуры данных представимые в виде DAG слоев. Будучи таковыми,
они могут быть безопасно сериализованы и десериализованы.

Наследованная модель отличается тем, что это не структура данных, это кусок кода. Архитектура модели определяется
в теле метода `call`. Это значит, что архитектура модели не может быть безопасно сериализована. Для загрузки модели вам понадобится доступ к коду который ее создал (код подкласса модели). Альтернативно, вы можете сериализовать этот код как байткод (e.g. via pickling), но это не безопасно и в общем не переносимо.

Для дополнительной информации об этих различиях смотри статью ["Что такое символьные и императивные API в TensorFlow 2.0?"](https://medium.com/tensorflow/what-are-symbolic-and-imperative-apis-in-tensorflow-2-0-dfccecb01021).

Давайте рассммотрим следующую наследованную модель, с той же структурой, что и модель в первой части:

In [ ]:
class ThreeLayerMLP(keras.Model):

  def __init__(self, name=None):
    super(ThreeLayerMLP, self).__init__(name=name)
    self.dense_1 = layers.Dense(64, activation='relu', name='dense_1')
    self.dense_2 = layers.Dense(64, activation='relu', name='dense_2')
    self.pred_layer = layers.Dense(10, activation='softmax', name='predictions')

  def call(self, inputs):
    x = self.dense_1(inputs)
    x = self.dense_2(x)
    return self.pred_layer(x)

def get_model():
  return ThreeLayerMLP(name='3_layer_mlp')

model = get_model()

Во-первых, *никогда не использовавшаяся наследованная модель не может быть сохранена*.

Это потому, что наследованная модель должна быть вызвана на некоторых данных, чтобы были созданы ее веса.

Пока модель не была вызвана, она не знает ожидаемые размерности и тип входных данных
и поэтому не может создать переменные весов. Вы можете помнить что в Functional модели из первой части, размеры и тип входных данных были определены заранее (с помощью `keras.Input(...)`) -- вот почему у Functional модели есть состояние сразу как создан ее экземпляр.

Давайте обучим модель, чтобы дать ей состояние:

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop())
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=1)

Рекомендованный способ сохранения наследованной модели это использование `save_weights` для создания чекпоинта TensorFlow SavedModel, который будет содержать значения всех переменных ассоциированных с моделью:
- Веса слоев
- Состояние оптимизатора
- Любые переменные связанные с метриками модели с состоянием (если такие есть)


In [ ]:
model.save_weights('path_to_my_weights', save_format='tf')

In [ ]:
# Сохраним прогнозы для проверки в будущем
predictions = model.predict(x_test)
# Также сохраним потери на первом пакете
# чтобы позже проверить что состояние оптимизатора было сохранено
first_batch_loss = model.train_on_batch(x_train[:64], y_train[:64])

Для восстановления вашей модели вам понадобится доступ к коду, который создал объект модели.

Обратите внимание, что для восстановления состояния оптимизатора или любой метрики с состоянием, вам нужно
скомпилировать модель (точно с теми же аргументами, что и раньше) и вызвать ее на каких-либо данных перед вызовом `load_weights`:

In [ ]:
# Восстановление модели
new_model = get_model()
new_model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.RMSprop())

# Здесь инициализируем переменные используемые отпимизаторами,
# так же как и переменные любых метрик с состоянием
new_model.train_on_batch(x_train[:1], y_train[:1])

# Загрузим состояние старой модели
new_model.load_weights('path_to_my_weights')

# Проверим что состояние сохранилось
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Состояние отпимизатораа также сохранилось,
# так что вы можете продолжить обучение с того места где остановились
new_first_batch_loss = new_model.train_on_batch(x_train[:64], y_train[:64])
assert first_batch_loss == new_first_batch_loss

Вы достигли конца этого руководства! Оно охватывает все, что вам нужно знать о сохранении и сериализации моделей с помощью tf.keras в TensorFlow 2.0.